In [1]:
from argparse import ArgumentParser, ArgumentDefaultsHelpFormatter

from addition_data import *
import sys
sys.path.insert(0,'../../')
import copy

#from texar.modules import TransformerEncoder, TransformerDecoder

#from rcalgo.tf.tfmodels.seq2seq import *
#import texar as tx
from collections import Counter
import random
import warnings
warnings.filterwarnings('ignore')

from rcalgo.tf.tfmodels.texar_models.tx_transformer_seq2seq import *
from rcalgo.tf.utils.text_utils import *
#from texar.utils import transformer_utils


In [2]:
#对联数据
dir_path = 'data/couplet/{}.txt'
with open(dir_path.format('in'), 'r') as f:
    source = f.readlines()
source = np.array([''.join(s.split()) for s in source])
with open(dir_path.format('out'), 'r') as f:
    target = f.readlines()
target = np.array([''.join(t.split()) for t in target])
maxlen = np.max([len(s) for s in source]) + 2

In [2]:
maxlen = 25
with open('data/chat_corpus/xiaohuangji.tsv', 'r') as f:
    contents = f.readlines()
    contents = [c.strip() for c in contents if len(c.strip().split('\t')) == 2]
source = np.array([c.split('\t')[0] for c in contents if len(c) <= maxlen*2+10])
target = np.array([c.split('\t')[1] for c in contents if len(c) <= maxlen*2+10])

In [3]:
word_freq = word_count(list(source) + list(target), word_seg=False)
term_id_map = build_word2idx(word_freq, freq_threshold=2)
id_term_map = dict([(v, k) for k,v in term_id_map.items()])

In [4]:
from rcalgo.tf.utils.train_config import *

args = parse_initial_training_args(
    [
        '--learning-rate', '0.0001',
       #'--nb-words', str(nwords),
        '--oov-buckets', '1',
        '--n-epochs', '4',
        '--opt', 'adam',
        '--batch-size', '128',
        '--p-coef', '0',
        '--n-head', '8',
        '--position-type','2,3,4,5',
        '--transformer-func','relu',
        '--keep-prob','0.8',
        '--embedding-size', '512',
        '--hidden-size', '512',
        '--gpu-id', '0',
        '--gpu-num', '4',
        '--expand-input',
        '--num-layers', '2',
        '--position-info',
        '--maxlen', str(maxlen)
    ]
)
args.output_maxlen = maxlen
args.beam_width = 200

In [5]:
graph_to_use = tf.Graph()
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth=True
seq2seq_model = TxTransformerSeq2SeqModel(args, graph_to_use, word_dict=term_id_map)
seq2seq_model.build_model()
seq2seq_model.build_model_summary()
display(seq2seq_model.model_summary())
#seq2seq_model.run([source[0:770400], target[0:770400]], test_size=0.2)


total parameter number: 26283520


variable_name  variable_shape  \
0     gpu/embedding/embedding_layer/embedding_table:0     [7686, 512]   
1            gpu/mw_conv_block/conv_1d/he_uniform/W:0   [2, 512, 512]   
2                       gpu/mw_conv_block/conv_1d/B:0           [512]   
3          gpu/mw_conv_block/conv_1d_1/he_uniform/W:0   [3, 512, 512]   
4                     gpu/mw_conv_block/conv_1d_1/B:0           [512]   
5          gpu/mw_conv_block/conv_1d_2/he_uniform/W:0   [4, 512, 512]   
6                     gpu/mw_conv_block/conv_1d_2/B:0           [512]   
7          gpu/mw_conv_block/conv_1d_3/he_uniform/W:0   [5, 512, 512]   
8                     gpu/mw_conv_block/conv_1d_3/B:0           [512]   
9                 gpu/pos_embedding/embedding_table:0       [34, 512]   
10  gpu/transform2_block/multi_head_attention/conv...  [1, 512, 1536]   
11  gpu/transform2_block/multi_head_attention/conv...          [1536]   
12  gpu/transform2_block/multi_head_attention/conv...   [1, 512, 512]   
13  gpu/transform2_block/multi_head_attention/conv...           [512]   
14               gpu/transform2_block/ln_layer/beta:0           [512]   
15              gpu/transform2_block/ln_layer/gamma:0           [512]   
16  gpu/transform2_block/ffn_mlp/conv_1d/he_unifor...  [1, 512, 2048]   
17           gpu/transform2_block/ffn_mlp/conv_1d/B:0          [2048]   
18  gpu/transform2_block/ffn_mlp/conv_1d_1/he_unif...  [1, 2048, 512]   
19         gpu/transform2_block/ffn_mlp/conv_1d_1/B:0           [512]   
20             gpu/transform2_block/ln_layer_1/beta:0           [512]   
21            gpu/transform2_block/ln_layer_1/gamma:0           [512]   
22  gpu/transform2_block_1/multi_head_attention/co...  [1, 512, 1536]   
23  gpu/transform2_block_1/multi_head_attention/co...          [1536]   
24  gpu/transform2_block_1/multi_head_attention/co...   [1, 512, 512]   
25  gpu/transform2_block_1/multi_head_attention/co...           [512]   
26             gpu/transform2_block_1/ln_layer/beta:0           [512]   
27            gpu/transform2_block_1/ln_layer/gamma:0           [512]   
28  gpu/transform2_block_1/ffn_mlp/conv_1d/he_unif...  [1, 512, 2048]   
29         gpu/transform2_block_1/ffn_mlp/conv_1d/B:0          [2048]   
..                                                ...             ...   
44  gpu/transformer_decoder/layer_0/encdec_attenti...      [512, 512]   
45  gpu/transformer_decoder/layer_0/encdec_attenti...      [512, 512]   
46  gpu/transformer_decoder/layer_0/encdec_attenti...      [512, 512]   
47  gpu/transformer_decoder/layer_0/past_poswise_l...           [512]   
48  gpu/transformer_decoder/layer_0/past_poswise_l...           [512]   
49  gpu/transformer_decoder/layer_0/ffn/conv1/kern...     [512, 2048]   
50   gpu/transformer_decoder/layer_0/ffn/conv1/bias:0          [2048]   
51  gpu/transformer_decoder/layer_0/ffn/conv2/kern...     [2048, 512]   
52   gpu/transformer_decoder/layer_0/ffn/conv2/bias:0           [512]   
53             gpu/transformer_decoder/layer_1/beta:0           [512]   
54            gpu/transformer_decoder/layer_1/gamma:0           [512]   
55  gpu/transformer_decoder/layer_1/self_attention...      [512, 512]   
56  gpu/transformer_decoder/layer_1/self_attention...      [512, 512]   
57  gpu/transformer_decoder/layer_1/self_attention...      [512, 512]   
58  gpu/transformer_decoder/layer_1/self_attention...      [512, 512]   
59  gpu/transformer_decoder/layer_1/encdec_attenti...           [512]   
60  gpu/transformer_decoder/layer_1/encdec_attenti...           [512]   
61  gpu/transformer_decoder/layer_1/encdec_attenti...      [512, 512]   
62  gpu/transformer_decoder/layer_1/encdec_attenti...      [512, 512]   
63  gpu/transformer_decoder/layer_1/encdec_attenti...      [512, 512]   
64  gpu/transformer_decoder/layer_1/encdec_attenti...      [512, 512]   
65  gpu/transformer_decoder/layer_1/past_poswise_l...           [512]   
66  gpu/transformer_decoder/layer_1/past_poswise_l...           [512]   
67  gpu/transformer_decod

In [6]:
with seq2seq_model.job.session.graph.as_default():
    input_var, context_var = tf.get_collection('input_dict')
    go_value = tf.get_collection('go_value')[0]
    outputs = tf.get_collection('output_dict')[0:5]
    greedy_logits, greedy_sample_id, sample_logits, sample_sample_id, beam_search_results = outputs
    top_k = tf.get_collection('top_k_sample')[0]
    softmax_temperature = tf.get_collection('softmax_temperature')[0]
    train_flag = tx.global_mode()


In [11]:
def out_print(num_arr, id_term_map, eos=seq2seq_model.nb_words - 1):
    #print(num_arr)
    text = ''
    for i in num_arr:
        if i >= eos-1:
            break
        else:
            text += id_term_map[i]
    return text

In [7]:
sents = np.random.choice(source, 20, replace=False)
sample_outputs, beam_search_outputs = seq2seq_model.job.session.run([sample_sample_id, beam_search_results], 
                                                                    feed_dict={input_var:sents, 
                                                                               train_flag:'infer'})
for i in range(len(sents)):
    print(sents[i])
    print('sample: {}'.format(out_print(sample_outputs[i], id_term_map)))
    print('beam search best: {}'.format(out_print(beam_search_outputs[i][:,0], id_term_map)))
    print()
